# Sandbox Container Setup Guide

## Interactive Notebook Version

This notebook provides an interactive way to set up your sandbox container environment. You can execute each code cell to follow along with the setup process.

**Note:** Some cells require manual intervention or confirmation. Read the comments carefully!

## Table of Contents
1. [Prerequisites Check](#Prerequisites-Check)
2. [Workspace Setup](#Workspace-Setup)
3. [Repository Setup](#Repository-Setup)
4. [Docker Configuration](#Docker-Configuration)
5. [Building and Running](#Building-and-Running)
6. [Verification](#Verification)
7. [Troubleshooting](#Troubleshooting)

## Prerequisites Check

Let's verify that all required tools are installed.

In [ ]:
# Check Docker installation
!docker --version

In [ ]:
# Check Docker Compose
!docker compose version

In [ ]:
# Check Git installation
!git --version

In [ ]:
# Test Docker by running hello-world
!docker run hello-world

### ✅ Prerequisites Checklist

Make sure all of the above commands executed successfully. If any failed:

- **Docker**: Install from [docker.com](https://docs.docker.com/get-docker/)
- **Git**: Install from [git-scm.com](https://git-scm.com/downloads)

Once everything is installed, re-run the cells above.

## Workspace Setup

Create a dedicated workspace for sandbox projects.

In [ ]:
import os
from pathlib import Path

# Define workspace path
# Change this if you want a different location
workspace_path = Path.home() / "sandbox-projects" / "pokemon-dev-sandbox"

print(f"Workspace will be created at: {workspace_path}")

# Create the workspace directory
workspace_path.mkdir(parents=True, exist_ok=True)

print(f"✅ Workspace created!")
print(f"   Location: {workspace_path}")

In [ ]:
# Change to workspace directory
%cd {workspace_path}
!pwd

## Repository Setup

Clone the repository and checkout the dev branch.

In [ ]:
# Clone the repository
# NOTE: This will fail if the directory already has a git repo
# If it fails, skip to the next cell

repo_url = "https://github.com/joesuuf/pokemon-dev.git"

!git clone {repo_url} .

print("\n✅ Repository cloned!")

In [ ]:
# List all available branches
print("Available branches:")
!git branch -a

In [ ]:
# Checkout the dev branch
dev_branch = "claude/check-tailwind-version-011CUfUBLNqZ6mnFTZtHHSZg"

!git checkout {dev_branch}

print(f"\n✅ Checked out branch: {dev_branch}")

In [ ]:
# Verify current branch and status
print("Current branch:")
!git branch

print("\nGit status:")
!git status

In [ ]:
# List project files
print("Project structure:")
!ls -la

## Docker Configuration

Create the necessary Docker configuration files.

### Create Dockerfile

The Dockerfile defines the container environment.

In [ ]:
%%writefile Dockerfile
# Use Node.js 18 Alpine (lightweight Linux distribution)
# Alpine is chosen for its small size (~5MB base) and security
FROM node:18-alpine

# Set the working directory inside the container
# All subsequent commands will run from this directory
WORKDIR /app

# Install essential tools
# git: Required for any git operations inside the container
# bash: Provides a better shell than Alpine's default sh
# python3, make, g++: Required for some npm packages with native dependencies
RUN apk add --no-cache \
    git \
    bash \
    python3 \
    make \
    g++

# Copy package files first (Docker layer caching optimization)
# By copying these separately, we can cache the npm install step
# This means faster rebuilds when only source code changes
COPY package*.json ./

# Install Node.js dependencies
# --frozen-lockfile ensures the lockfile isn't modified
# This guarantees consistent dependency versions
RUN npm install --frozen-lockfile

# Copy the rest of the application
# Done after npm install to leverage Docker's layer caching
COPY . .

# Expose the development server port
# This makes the port available to your host machine
EXPOSE 3000

# Set environment to development
ENV NODE_ENV=development

# Default command when container starts
# Can be overridden with docker run command
CMD ["npm", "run", "dev"]

In [ ]:
# Verify Dockerfile was created
!cat Dockerfile

### Create docker-compose.yml

Docker Compose makes managing containers easier.

In [ ]:
%%writefile docker-compose.yml
version: '3.8'

services:
  # Main development service
  dev:
    # Build from the local Dockerfile
    build:
      context: .
      dockerfile: Dockerfile

    # Container name (easier to reference)
    container_name: pokemon-dev-sandbox

    # Port mapping: host:container
    # Access the app at localhost:3000
    ports:
      - "3000:3000"

    # Volume mounts for live code editing
    # Changes on your host immediately reflect in the container
    volumes:
      - .:/app                          # Mount project directory
      - /app/node_modules               # Preserve container's node_modules

    # Environment variables
    environment:
      - NODE_ENV=development
      - VITE_API_KEY=${VITE_API_KEY}

    # Restart policy
    restart: unless-stopped

    # Keep container running
    stdin_open: true
    tty: true

    # Override default command for development
    command: npm run dev

  # Optional: Test runner service
  test:
    build:
      context: .
      dockerfile: Dockerfile
    container_name: pokemon-dev-test
    volumes:
      - .:/app
      - /app/node_modules
    environment:
      - NODE_ENV=test
    command: npm run test:watch
    profiles:
      - testing  # Only starts with --profile testing

  # Optional: Build verification service
  build:
    build:
      context: .
      dockerfile: Dockerfile
    container_name: pokemon-dev-build
    volumes:
      - .:/app
      - /app/node_modules
    command: npm run build
    profiles:
      - build  # Only starts with --profile build

In [ ]:
# Verify docker-compose.yml was created
!cat docker-compose.yml

### Create .dockerignore

Prevent unnecessary files from being copied into Docker.

In [ ]:
%%writefile .dockerignore
# Dependencies
node_modules
npm-debug.log*
yarn-debug.log*
yarn-error.log*

# Testing
coverage
.nyc_output

# Build output
dist
build

# Environment files
.env
.env.local
.env.*.local

# IDE
.vscode
.idea
*.swp
*.swo
*~

# OS
.DS_Store
Thumbs.db

# Git
.git
.gitignore
.gitattributes

# Docker
Dockerfile
docker-compose.yml
.dockerignore

# Documentation
*.md
docs/

# CI/CD
.github
.gitlab-ci.yml

### Create Environment File

**⚠️ IMPORTANT:** Add your actual API keys after creating this file!

In [ ]:
%%writefile .env
# Development Environment Variables
NODE_ENV=development

# Vite Configuration
VITE_PORT=3000

# API Keys (REPLACE WITH YOUR ACTUAL KEYS)
VITE_API_KEY=your-pokemon-tcg-api-key-here

# Optional: Enable debug mode
DEBUG=true
VITE_DEBUG=true

In [ ]:
# Create .env.example for documentation
!cp .env .env.example

print("✅ Environment files created!")
print("⚠️  REMEMBER: Edit .env and add your actual API keys!")

## Building and Running

Now let's build and run the container!

### Build the Docker Image

**Note:** This can take 2-5 minutes depending on your internet speed.

In [ ]:
# Build the Docker image
!docker build -t pokemon-dev-sandbox .

print("\n✅ Docker image built successfully!")

In [ ]:
# Verify the image was created
!docker images | grep pokemon-dev-sandbox

### Start the Container with Docker Compose

**Note:** This will start the container in detached mode (background).

In [ ]:
# Start the container in detached mode
!docker compose up -d

print("\n✅ Container started!")
print("   Access your app at: http://localhost:3000")

In [ ]:
# Check container status
!docker compose ps

In [ ]:
# View container logs (last 50 lines)
!docker compose logs --tail=50

## Verification

Let's verify everything is working correctly.

In [ ]:
# Check if the server is responding
import requests
import time

print("Waiting for server to start...")
time.sleep(5)  # Give the server time to start

try:
    response = requests.get("http://localhost:3000", timeout=10)
    if response.status_code == 200:
        print("✅ Server is running!")
        print(f"   Status Code: {response.status_code}")
        print(f"   Open http://localhost:3000 in your browser")
    else:
        print(f"⚠️  Server responded with status code: {response.status_code}")
except Exception as e:
    print(f"❌ Could not connect to server: {e}")
    print("   Check the logs above for errors")

In [ ]:
# Execute command inside the container
print("Node.js version in container:")
!docker exec pokemon-dev-sandbox node --version

print("\nnpm version in container:")
!docker exec pokemon-dev-sandbox npm --version

In [ ]:
# List installed packages
print("Installed npm packages:")
!docker exec pokemon-dev-sandbox npm list --depth=0

## Working with the Container

Here are common operations you'll perform.

In [ ]:
# View real-time logs (use Ctrl+C to stop in terminal)
# In Jupyter, this will show last 20 lines
!docker compose logs --tail=20 -f &

In [ ]:
# Restart the container
!docker compose restart

print("✅ Container restarted")

In [ ]:
# Stop the container
!docker compose stop

print("✅ Container stopped")

In [ ]:
# Start the container again
!docker compose start

print("✅ Container started")

## Running Commands in Container

In [ ]:
# Run tests
!docker exec pokemon-dev-sandbox npm run test

In [ ]:
# Run build
!docker exec pokemon-dev-sandbox npm run build

In [ ]:
# Check git status inside container
!docker exec pokemon-dev-sandbox git status

## Troubleshooting

Use these cells if you encounter issues.

In [ ]:
# Check if port 3000 is in use
!lsof -i :3000 || echo "Port 3000 is available"

In [ ]:
# View detailed container logs
!docker compose logs --tail=100

In [ ]:
# Inspect container configuration
!docker inspect pokemon-dev-sandbox

In [ ]:
# Rebuild without cache (if having build issues)
!docker compose build --no-cache

In [ ]:
# Check Docker disk usage
!docker system df

In [ ]:
# Clean up Docker resources (if running low on disk space)
!docker system prune -f

print("✅ Docker resources cleaned up")

## Cleanup and Shutdown

In [ ]:
# Stop and remove containers (keeps images and volumes)
!docker compose down

print("✅ Containers stopped and removed")

In [ ]:
# Stop, remove containers, and remove volumes
# ⚠️ WARNING: This will delete any data stored in volumes
!docker compose down -v

print("✅ Containers and volumes removed")

In [ ]:
# Remove the Docker image
# ⚠️ WARNING: You'll need to rebuild if you want to use it again
!docker rmi pokemon-dev-sandbox

print("✅ Docker image removed")

## Quick Reference

### Common Commands

```bash
# Start
docker compose up -d

# Stop
docker compose down

# Logs
docker compose logs -f

# Restart
docker compose restart

# Execute command
docker exec pokemon-dev-sandbox <command>

# Shell access
docker exec -it pokemon-dev-sandbox bash

# Rebuild
docker compose build --no-cache
```

## Next Steps

1. ✅ Open http://localhost:3000 in your browser
2. 📚 Review the Tailwind CSS guide: `tailwind-v4-guide.ipynb`
3. 🤖 Review the Claude Code guide: `anthropic-claude-code-guide.ipynb`
4. 🚀 Start experimenting safely in your sandbox!

---

**Happy Sandboxing! 🎉**